# 1. Define a research question:

### Which community districts in New York City show the highest number of 311 complaints?

# 2. What data are available?

### 311 Service Requests from 2010 to the Present, https://datahub.cusp.nyu.edu/dataset/erm2-nwe9

In [53]:
# import packages
import os
import pandas as pd
import numpy as np

In [3]:
# import dataset
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
complaints = data
print (complaints.columns)
complaints.head()

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11224,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,NaN,NaN,NaN,1-1-1173130914,NaN,NaN,NaN,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,Street,11234,1157 EAST 57 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11218,722 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Think about possible measurement errors

In [22]:
# What are the unique values for community districts?

CommDist = complaints['Community Board'].unique()

In [49]:
print ('There are %i unique values for community board in this dataset.' %len(CommDist))
print (CommDist)

There are 77 unique values for community board in this dataset.
['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS' '09 BROOKLYN'
 'Unspecified MANHATTAN' 'Unspecified BROOKLYN' '07 BROOKLYN'
 'Unspecified BRONX' '64 MANHATTAN' '81 QUEENS' '5

### There are 59 community districts in NYC, but this dataset has 77 unique values for community district

In [19]:
CommDist.sort()
print (CommDist)

['0 Unspecified' '01 BRONX' '01 BROOKLYN' '01 MANHATTAN' '01 QUEENS'
 '01 STATEN ISLAND' '02 BRONX' '02 BROOKLYN' '02 MANHATTAN' '02 QUEENS'
 '02 STATEN ISLAND' '03 BRONX' '03 BROOKLYN' '03 MANHATTAN' '03 QUEENS'
 '03 STATEN ISLAND' '04 BRONX' '04 BROOKLYN' '04 MANHATTAN' '04 QUEENS'
 '05 BRONX' '05 BROOKLYN' '05 MANHATTAN' '05 QUEENS' '06 BRONX'
 '06 BROOKLYN' '06 MANHATTAN' '06 QUEENS' '07 BRONX' '07 BROOKLYN'
 '07 MANHATTAN' '07 QUEENS' '08 BRONX' '08 BROOKLYN' '08 MANHATTAN'
 '08 QUEENS' '09 BRONX' '09 BROOKLYN' '09 MANHATTAN' '09 QUEENS' '10 BRONX'
 '10 BROOKLYN' '10 MANHATTAN' '10 QUEENS' '11 BRONX' '11 BROOKLYN'
 '11 MANHATTAN' '11 QUEENS' '12 BRONX' '12 BROOKLYN' '12 MANHATTAN'
 '12 QUEENS' '13 BROOKLYN' '13 QUEENS' '14 BROOKLYN' '14 QUEENS'
 '15 BROOKLYN' '16 BROOKLYN' '17 BROOKLYN' '18 BROOKLYN' '26 BRONX'
 '27 BRONX' '28 BRONX' '55 BROOKLYN' '56 BROOKLYN' '64 MANHATTAN'
 '80 QUEENS' '81 QUEENS' '82 QUEENS' '83 QUEENS' '84 QUEENS'
 '95 STATEN ISLAND' 'Unspecified BRONX' 'Unsp

## The 3 districts for Staten Island, 12 each for Bronx and Manhattan, 14 for Queens, and 18 for Brooklyn are included above (https://www1.nyc.gov/site/planning/community/community-portal.page). The community districts numbered above 18, and "0 Unspecified", are all inaccurate.

In [39]:
print ("There are %i complaints in this dataset." %len(complaints))
print ("There are %i unique keys for complaints in this dataset." %len(complaints['Unique Key'].unique()))
print ("There are fewer 'Unique Key' values than total rows in the dataset, meaning you have some duplicate complaints.")

There are 10187766 complaints in this dataset.
There are 10187744 unique keys for complaints in this dataset.
There are fewer 'Unique Key' values than total rows in the dataset, meaning you have some duplicate complaints.


In [104]:
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified MANHATTAN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: 
                                                                  x.replace('Unspecified STATEN ISLAND', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('26 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('27 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('28 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('55 BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('56 BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('64 MANHATTAN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('80 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('81 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('82 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('83 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('84 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('95 STATEN ISLAND', 'Unspecified'))

CommDist = complaints['Community Board'].unique()
CommDist.sort()
print ('There are now %i unique values for community board in this dataset.' %len(CommDist))
print (CommDist)

There are 60 unique values for community board in this dataset.
['01 BRONX' '01 BROOKLYN' '01 MANHATTAN' '01 QUEENS' '01 STATEN ISLAND'
 '02 BRONX' '02 BROOKLYN' '02 MANHATTAN' '02 QUEENS' '02 STATEN ISLAND'
 '03 BRONX' '03 BROOKLYN' '03 MANHATTAN' '03 QUEENS' '03 STATEN ISLAND'
 '04 BRONX' '04 BROOKLYN' '04 MANHATTAN' '04 QUEENS' '05 BRONX'
 '05 BROOKLYN' '05 MANHATTAN' '05 QUEENS' '06 BRONX' '06 BROOKLYN'
 '06 MANHATTAN' '06 QUEENS' '07 BRONX' '07 BROOKLYN' '07 MANHATTAN'
 '07 QUEENS' '08 BRONX' '08 BROOKLYN' '08 MANHATTAN' '08 QUEENS' '09 BRONX'
 '09 BROOKLYN' '09 MANHATTAN' '09 QUEENS' '10 BRONX' '10 BROOKLYN'
 '10 MANHATTAN' '10 QUEENS' '11 BRONX' '11 BROOKLYN' '11 MANHATTAN'
 '11 QUEENS' '12 BRONX' '12 BROOKLYN' '12 MANHATTAN' '12 QUEENS'
 '13 BROOKLYN' '13 QUEENS' '14 BROOKLYN' '14 QUEENS' '15 BROOKLYN'
 '16 BROOKLYN' '17 BROOKLYN' '18 BROOKLYN' 'Unspecified']


    Need to finish this part --> drop duplicates, then group by  commboard

In [107]:
#complaints.drop_duplicates('Unique Key', keep='first')

# 4. Think about the interpretation of your results.

In [54]:
pop = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')

In [55]:
pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [109]:
cd_id = pop['cd_id'].unique()
print (cd_id)

['BRONX 08' 'BRONX 12' 'BRONX 10' 'BRONX 11' 'BRONX 03' 'BRONX 06'
 'BRONX 07' 'BRONX 05' 'BRONX 04' 'BRONX 09' 'BRONX 01' 'BRONX 02'
 'MANHATTAN 12' 'MANHATTAN 09' 'MANHATTAN 10' 'MANHATTAN 11 '
 'MANHATTAN 08' 'MANHATTAN 07' 'MANHATTAN 05' 'MANHATTAN 04' 'MANHATTAN 06'
 'MANHATTAN 03' 'MANHATTAN 01' 'MANHATTAN 02' 'STATEN ISLAND 03'
 'STATEN ISLAND 02' 'STATEN ISLAND 01' 'BROOKLYN 01' 'BROOKLYN 04'
 'BROOKLYN 03' 'BROOKLYN 02' 'BROOKLYN 06' 'BROOKLYN 08' 'BROOKLYN 16'
 'BROOKLYN 05' 'BROOKLYN 18' 'BROOKLYN 17' 'BROOKLYN 09' 'BROOKLYN 07'
 'BROOKLYN 10' 'BROOKLYN 12' 'BROOKLYN 14' 'BROOKLYN 15' 'BROOKLYN 11'
 'BROOKLYN 13' 'QUEENS 01' 'QUEENS 03' 'QUEENS 07' 'QUEENS 11' 'QUEENS 13'
 'QUEENS 08' 'QUEENS 04' 'QUEENS 06' 'QUEENS 02' 'QUEENS 05' 'QUEENS 09'
 'QUEENS 12' 'QUEENS 10' 'QUEENS 14']


In [114]:
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('BX', 'BRONX '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('MN', 'MANHATTAN '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('SI', 'STATEN_ISLAND '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('BK', 'BROOKLYN '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('QN', 'QUEENS '))
cd_id = pop['cd_id'].unique()
print (pop['cd_id'].unique())

['BRONX 08' 'BRONX 12' 'BRONX 10' 'BRONX 11' 'BRONX 03' 'BRONX 06'
 'BRONX 07' 'BRONX 05' 'BRONX 04' 'BRONX 09' 'BRONX 01' 'BRONX 02'
 'MANHATTAN 12' 'MANHATTAN 09' 'MANHATTAN 10' 'MANHATTAN 11 '
 'MANHATTAN 08' 'MANHATTAN 07' 'MANHATTAN 05' 'MANHATTAN 04' 'MANHATTAN 06'
 'MANHATTAN 03' 'MANHATTAN 01' 'MANHATTAN 02' 'STATEN_ISLAND 03'
 'STATEN_ISLAND 02' 'STATEN_ISLAND 01' 'BROOKLYN 01' 'BROOKLYN 04'
 'BROOKLYN 03' 'BROOKLYN 02' 'BROOKLYN 06' 'BROOKLYN 08' 'BROOKLYN 16'
 'BROOKLYN 05' 'BROOKLYN 18' 'BROOKLYN 17' 'BROOKLYN 09' 'BROOKLYN 07'
 'BROOKLYN 10' 'BROOKLYN 12' 'BROOKLYN 14' 'BROOKLYN 15' 'BROOKLYN 11'
 'BROOKLYN 13' 'QUEENS 01' 'QUEENS 03' 'QUEENS 07' 'QUEENS 11' 'QUEENS 13'
 'QUEENS 08' 'QUEENS 04' 'QUEENS 06' 'QUEENS 02' 'QUEENS 05' 'QUEENS 09'
 'QUEENS 12' 'QUEENS 10' 'QUEENS 14']


In [61]:
print ("There are %i unique community districts in this dataset" %len(pop['cd_id'].unique()))

There are 59 unique community districts in this dataset


In [64]:
cd_id.sort()
print (cd_id)

['BRONX 01' 'BRONX 02' 'BRONX 03' 'BRONX 04' 'BRONX 05' 'BRONX 06'
 'BRONX 07' 'BRONX 08' 'BRONX 09' 'BRONX 10' 'BRONX 11' 'BRONX 12'
 'BROOKLYN 01' 'BROOKLYN 02' 'BROOKLYN 03' 'BROOKLYN 04' 'BROOKLYN 05'
 'BROOKLYN 06' 'BROOKLYN 07' 'BROOKLYN 08' 'BROOKLYN 09' 'BROOKLYN 10'
 'BROOKLYN 11' 'BROOKLYN 12' 'BROOKLYN 13' 'BROOKLYN 14' 'BROOKLYN 15'
 'BROOKLYN 16' 'BROOKLYN 17' 'BROOKLYN 18' 'MANHATTAN 01' 'MANHATTAN 02'
 'MANHATTAN 03' 'MANHATTAN 04' 'MANHATTAN 05' 'MANHATTAN 06' 'MANHATTAN 07'
 'MANHATTAN 08' 'MANHATTAN 09' 'MANHATTAN 10' 'MANHATTAN 11111'
 'MANHATTAN 12' 'QUEENS 01' 'QUEENS 02' 'QUEENS 03' 'QUEENS 04' 'QUEENS 05'
 'QUEENS 06' 'QUEENS 07' 'QUEENS 08' 'QUEENS 09' 'QUEENS 10' 'QUEENS 11'
 'QUEENS 12' 'QUEENS 13' 'QUEENS 14' 'STATEN ISLAND 01' 'STATEN ISLAND 02'
 'STATEN ISLAND 03']


In [69]:
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('MANHATTAN 11111', 'MANHATTAN 11'))
cd_id = pop['cd_id'].unique()
cd_id.sort()
print (cd_id)

['BRONX 01' 'BRONX 02' 'BRONX 03' 'BRONX 04' 'BRONX 05' 'BRONX 06'
 'BRONX 07' 'BRONX 08' 'BRONX 09' 'BRONX 10' 'BRONX 11' 'BRONX 12'
 'BROOKLYN 01' 'BROOKLYN 02' 'BROOKLYN 03' 'BROOKLYN 04' 'BROOKLYN 05'
 'BROOKLYN 06' 'BROOKLYN 07' 'BROOKLYN 08' 'BROOKLYN 09' 'BROOKLYN 10'
 'BROOKLYN 11' 'BROOKLYN 12' 'BROOKLYN 13' 'BROOKLYN 14' 'BROOKLYN 15'
 'BROOKLYN 16' 'BROOKLYN 17' 'BROOKLYN 18' 'MANHATTAN 01' 'MANHATTAN 02'
 'MANHATTAN 03' 'MANHATTAN 04' 'MANHATTAN 05' 'MANHATTAN 06' 'MANHATTAN 07'
 'MANHATTAN 08' 'MANHATTAN 09' 'MANHATTAN 10' 'MANHATTAN 11 '
 'MANHATTAN 12' 'QUEENS 01' 'QUEENS 02' 'QUEENS 03' 'QUEENS 04' 'QUEENS 05'
 'QUEENS 06' 'QUEENS 07' 'QUEENS 08' 'QUEENS 09' 'QUEENS 10' 'QUEENS 11'
 'QUEENS 12' 'QUEENS 13' 'QUEENS 14' 'STATEN ISLAND 01' 'STATEN ISLAND 02'
 'STATEN ISLAND 03']


In [115]:
pop['commdist'] = None
for i in range (0, len(pop)):
    pop['commdist'][i] = pop['cd_id'][i].split(' ')[1] + ' ' + pop['cd_id'][i].split(' ')[0]
print (pop['commdist'].unique())

['08 BRONX' '12 BRONX' '10 BRONX' '11 BRONX' '03 BRONX' '06 BRONX'
 '07 BRONX' '05 BRONX' '04 BRONX' '09 BRONX' '01 BRONX' '02 BRONX'
 '12 MANHATTAN' '09 MANHATTAN' '10 MANHATTAN' '11 MANHATTAN' '08 MANHATTAN'
 '07 MANHATTAN' '05 MANHATTAN' '04 MANHATTAN' '06 MANHATTAN' '03 MANHATTAN'
 '01 MANHATTAN' '02 MANHATTAN' '03 STATEN_ISLAND' '02 STATEN_ISLAND'
 '01 STATEN_ISLAND' '01 BROOKLYN' '04 BROOKLYN' '03 BROOKLYN' '02 BROOKLYN'
 '06 BROOKLYN' '08 BROOKLYN' '16 BROOKLYN' '05 BROOKLYN' '18 BROOKLYN'
 '17 BROOKLYN' '09 BROOKLYN' '07 BROOKLYN' '10 BROOKLYN' '12 BROOKLYN'
 '14 BROOKLYN' '15 BROOKLYN' '11 BROOKLYN' '13 BROOKLYN' '01 QUEENS'
 '03 QUEENS' '07 QUEENS' '11 QUEENS' '13 QUEENS' '08 QUEENS' '04 QUEENS'
 '06 QUEENS' '02 QUEENS' '05 QUEENS' '09 QUEENS' '12 QUEENS' '10 QUEENS'
 '14 QUEENS']


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
